In [ ]:
#Implement controllers
#Implement controller-estimators
#Plot with step response
#Plot with different trajectories
#Compute Tracking error of all models plot
#Compute Estimation error for all KF's

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Implements feedback loop for controller and estimator
class ControllerEstimator:
    def __init__(self, controller, estimator):
        self.controller = controller
        self.estimator = estimator

    #initialize controller and observer
    def initialize(self, state=None, control=None):
        self.controller.initialize(self, state, control)
        self.estimator.initialize(self, state, control)

    #Take state and target and return control signal
    def step(self, state, target):
       #Get state estimate
        estimation = self.estimator.step(state)

        #get control signal given state estimate
        error = target - estimation
        control = self.controller.step(error)

        return control
    
    #get trajectory
    def getTrajectory(self):
        return self.estimator.getTrajectory()

In [ ]:
#Template class for controllers
class LinearController:
    def __init__(self, dt, mingain=-float("inf"), maxgain=float("inf")):
        #set timestep
        self.dt = dt

        #define min and max gains
        self.mingain = mingain
        self.maxgain = maxgain

    #Change timestep
    def dtUpdate(self, dt):
        self.dt = dt

    #Initialize controller
    def initialize(self, state=None, control=None)
        pass

    #Run step of controller given error
    def step(self, error, equilibrium=None):
        pass

In [ ]:
class SISO_CONTROLLER_TYPE:
    PID = 0
    DPID = 1
    LLC = 2

#Proportional Integral Derivative Controller
class ProportionalIntegralDerivative(LinearController):
    def __init__(self, kp, ki, kd, dt, mingain=-float("inf"), maxgain=float("inf")):
        super.__init__(self, dt, mingain, maxgain)
        self.kp = kp
        self.ki = ki
        self.kd = kd

    def step(self, error):
        #D: compute dedt
        if self.e_last is not None:
            dedt = (error - self.e_last) / self.dt
        else:
            dedt = 0

        #I: compute integral
        self.int_e += error * self.dt

        #PID: compute gain
        gain = (self.kp * error) + (self.ki * self.int_e) + (self.kd * dedt)
        self.e_last = error

        return min(max(self.mingain, gain), self.maxgain)
    
#Proportional Integral Derivative Controller
#   -computes PID in discrete space using bilinear transform
class DiscreteProportionalIntegralDerivative(LinearController):
    def __init__(self, kp, ki, kd, dt, mingain=-float("inf"), maxgain=float("inf")):
        super.__init__(self, dt, mingain, maxgain)

        #define parameter from kp, kd, ki
        self.k = kp + (2 * ki) / self.dt + (0.5 * ki * self.dt)

        #define previous errors
        self.e1 = None
        self.e2 = None

        #define previous controls
        self.c1 = None
        self.c2 = None

    def step(self, error):
        #compute gain
        gain = (self.k * error) + (2 * self.k * self.e1) + (self.k * self.e2) - self.c2

        #store 2nd error and control signals
        if self.e1 and self.c1:
            self.e2 = self.e1
            self.c2 = self.c1

        #define 1st error and control signals
        self.e1 = error
        self.c1 = gain

        return min(max(self.mingain, gain), self.maxgain)
    
#Lead-Lag Compensator with params [K0, K1, K2, K3, gamma]
class LeadLagCompensator(LinearController):
    def __init__(self, k, dt, mingain=-float("inf"), maxgain=float("inf"), discrete=True):
        super.__init__(self, dt, mingain, maxgain)

        #define parameters
        self.k = k

        #define previous errors
        self.e1 = None
        self.e2 = None

        #define previous controls
        self.c1 = None
        self.c2 = None

    #Run one step of the general lead lag compensator
    #   -derived by taking the LLC transfer function and discretizing with bilinear transform
    def step(self, error):
        #define terms
        a = (4 / self.dt**2)
        b = (2 * self.k[3]) / (self.dt * self.k[4])
        c = (2 * self.k[4]) / (self.dt * self.k[2])
        d = 2 / (self.dt * self.k[0])
        e = 2 / (self.dt * self.k[1])

        #define parameters
        A = a + b + c
        B = 8 / self.dt**2
        C = a + b + c + self.k[2] * self.k[3]
        D = a + d + e
        E = a + d + e + 1 / (self.k[0] ( self.k[1]))

        #compute LLC control signal
        gain = ((E / C) * error) + ((B / C) * (self.e1 - self.c1)) + ((D / C) * self.e2) - ((A / C) * self.c2)

        #store error and control signals
        if not self.e1 or not self.c1:
            self.e1 = error
            self.c1 = gain
        else:
            self.e2 = self.e1
            self.c2 = self.c1
            self.e1 = error
            self.c1= gain

        return min(max(self.mingain, gain), self.maxgain)

In [ ]:
#Cascaded PID for Planar Dynamical Systems with q = [x,y,theta]
class CascadedPlanarController(LinearController):
    def __init__(self, K, dt, mingain=-float("inf"), maxgain=float("inf"), ctype=SISO_CONTROLLER_TYPE.PID):
        super.__init__(self, dt, mingain, maxgain)
        assert K.shape[0] > 4, "Gain matrix too large"

        #check for type of cascade (thrust coupling???): [x,y,thetaX, thetaY]
        for i in range(K.shape[0]):
            #Check for controller type
            if ctype == SISO_CONTROLLER_TYPE.PID:
                self.controllers[i] = ProportionalIntegralDerivative(K[i], self.dt, self.mingain, self.maxgain)
            elif ctype == SISO_CONTROLLER_TYPE.DPID:
                self.controllers[i] = DiscreteProportionalIntegralDerivative(K[i], self.dt, self.mingain, self.maxgain)
            elif ctype == SISO_CONTROLLER_TYPE.LLC:
                self.controllers[i] = LeadLagCompensator(K[i], self.dt, self.mingain, self.maxgain)

    #Change timestep
    def dtUpdate(self, dt):
        self.dt = dt
        for controller in self.controllers:
            controller.dtUpdate(self.dt)

    #Define cascading
    def step(self, error):
        #Coupled vs. uncoupled thrust
        gain = 0.0
        if len(self.controllers) == 3:
            #compute x and y controllers
            target_theta = self.controllers[0].step(error[0])
            thrust = self.controllers[2].step(error[1])

            #compute theta controller
            dthrust = self.controllers[2].step(target_theta - error[2])

            #compute gain
            gain = np.array([thrust - dthrust, thrust + dthrust])
        else:
            #compute x and y controllers
            target_theta = self.controllers[0].step(error[0])
            thrust = self.controllers[2].step(error[1])

            #compute theta controller
            dthrustx = self.controllers[2].step(target_theta - error[2])
            dthrusty = self.controllers[3].step(target_theta - error[2])

            #compute gain
            gain = np.array([thrust - dthrustx, thrust + dthrusty])

        return gain

In [ ]:
#Cascaded Lead/Lag Compensator Response


In [ ]:
#Cascaded Discrete Lead/Lag Compensator Response


In [ ]:
#Pole Placement Response


In [ ]:
#Controllers with Kalman Filter (Weiner model)


In [ ]:
#Controllers with Kalman Filter (Linearized model)


In [ ]:
#Controllers with Extended Kalman Filter
